# **Procedural SQL**

This notebook will cover procedural SQL topics such as stored procedures, functions, triggers, and views. These tools help automate tasks, ensure referential integrity, and retrieve processed data.

## **Stored Procedures**

A stored procedure is a set of SQL statements that can be executed as a single unit, kind of like a transaction. Stored procedures are stored in the database and can be invoked by name. They allow for modular, reusable, and maintainable code. In SQL Server, stored procedures can accept parameters, perform conditional logic, and return values. Let's start by creating a stored procedure that retrieves all records from the gs\_products table in the guitar store database.

To start we will use the key words CREATE PROCEDURE and then give the procedure a name. We specify the contents of the procedure using AS, and enclose the script of the procedure in BEGIN/END:

In [ ]:
--Create stored procedure to retrieve all records from gs_products
CREATE PROCEDURE get_all_products
AS
BEGIN
    SELECT * FROM gs_products
END;

When we run the above command, the response indicates success, but there is no returned data. This is because we have not executed the procedure, we have just saved it in the database. In the connections pane (top left icon) follow these instructions:

1. Expand the connected database so you can see it's folders
2. Open the databases folder
3. Navigate to your student database
4. Open the Programmability folder
5. Open the Stored Procedures folder

After following the above steps you should see the procedure that we just created. It is important that you know where to find these in case you ever forget the name of a procedure. This procedure can be executed usin the EXEC keyword. Let's run it now:

In [ ]:
--Execute get_all_products stored procedure
EXEC get_all_products;

We can see that the stored procedure executes the script that we saved earlier. This makes them useful for running repettive tasks or executing a SQL script from a different application. We can make our stored procedures more powerful by implementing _parameters_. Parameters are similar to variables. We can use them to store a value for use in the procedure. We instantiate them using the @ symbol and specifying their data type. We are going to create a new procedure that selects all products that have a list\_price higher than the provided parameter.

In [ ]:
--Create stored procedure to retrieve all records from gs_products with list_price higher than parameter
CREATE PROCEDURE get_all_products_higher_than
    @list_price_min DECIMAL
AS
BEGIN
    SELECT *
    FROM gs_products
    WHERE list_price > @list_price_min
END;

Now that we have saved the procedure, we can execute it and provide a value for the parameter:

In [ ]:
EXEC get_all_products_higher_than @list_price_min = 500

We can see that the procedure returns all products with a list\_price higher than the value provided to the parameter. Try changing 500 to another value and re-running the cell. You can see that the return updates dynamically without any changes to the procedure itself.

  

Parameters make our scripts more powerful because they give us a way to make script that is flexible and dynamic without ever having to change the actual source script. When an application is interacting with a database it is typically calling a stored procedure and passing paramaters into it, not using full SQL scripts to make changes or selections.

  

We can also use parameters to return values when using a stored procedure. The below procedure returns the lowest priced product that is more expensive than the price passe din via parameter (if a customer wants to spend at least 500, but wants the cheapest item that is over 500 then we can get that). We do this by specifying that a parameter is for OUTPUT:

In [ ]:
--Create stored procedure to retrieve product from gs_products with lowest list_price higher than parameter
CREATE PROCEDURE get_cheapest_product_higher_than
    @list_price_min DECIMAL,
    @product_name_cheap VARCHAR(50) OUTPUT --Specify parameter as an output
AS
BEGIN
    --We specify the parameter in the select statement to setup the return
    SELECT @product_name_cheap = product_name
    FROM gs_products
    WHERE list_price = (
        --Subquery to get minimum price of product that is higher than provided price
        --Making DISTINCT to ensure only one value is returned
        SELECT DISTINCT MIN(list_price)
        FROM gs_products
        WHERE list_price > @list_price_min
    )
END;

Now that the procedure has been saved we can execute it.

In [ ]:
-- Declare a variable to hold the output parameter value
DECLARE @product VARCHAR(50);

--Execute the procedure
EXEC get_cheapest_product_higher_than @list_price_min = 500, @product_name_cheap = @product OUTPUT;

--Print the result
PRINT @product;

Finally, we can write our stored procedures using conditional logic so that we can perform different actions based on the value of a parameter that has been passed in. Int he below example we are going to create a stored procedure that returns the name and price of a product based on the id number provided by the parameter. If the id number does not match one in the products table, then we will return an error. Otherwise, we will execute the procedure normally.

In [ ]:
--Create stored procedure to return name and price of product based on ID
CREATE PROCEDURE get_product_info_by_id
    @product_id INT, --parameter containing product id to search for
    @product_info VARCHAR(50) OUTPUT --parameter to return name and price
AS
BEGIN
    IF @product_id NOT IN (SELECT product_id FROM gs_products) --IF statement to check if provided id is in list of id's from table
    BEGIN
        SELECT @product_info = 'Please provide a valid Product ID'; --Select statement to return an error message
    END
    ELSE --ELSE keyword paired with IF. Runs if the condition in the IF statement is false
    BEGIN
        SELECT @product_info = product_name + ' ' + CAST(list_price AS VARCHAR(10)) --Casting the list_price to a VARCHAR so that it can be concatenated with name and returned as one value
        FROM gs_products
        WHERE product_id = @product_id; --Condition to filter result to only products where the id matches the parameter
    END
END;

The above script seems complicated on the surface, but it is only implementing the tools that we already understand (or will understand soon) into a conditional statement using IF/ELSE. If we call the stored procedure with an acceptable value, like 5, then we should get back the requested information:

In [ ]:
-- Declare a variable to hold the output parameter value
DECLARE @product VARCHAR(50);

--Execute the procedure
EXEC get_product_info_by_id @product_id = 5, @product_info = @product OUTPUT;

--Print the result
PRINT @product;

The above cell prints out the requested data with no problem. We are going to execute the same procedure again, but instead of 5, we will use a value that shouldn't work, like 15.

In [ ]:
-- Declare a variable to hold the output parameter value
DECLARE @product VARCHAR(50);

--Execute the procedure
EXEC get_product_info_by_id @product_id = 15, @product_info = @product OUTPUT;

--Print the result
PRINT @product;

We can see that this time we are getting the error message that we programmed earlier. When designing real applications, the interaction with the database would look something like this for a new customer signing up:

1. User fills out sign-up form in app
2. Data is stored in an object/variables in the app
3. The app connects to the database and passes the user info into parameters of a stored procedure
4. The stored procedure runs an insert transaction on all applicable tables, and then returns an indicator of error or success to the app
5. The app provides feedback to the user

This is a very simplified overview, but the take-away is that we aren't typically just running SQL from an application. We should always call a procedure and pass in any needed information. This is crucial for security and referenctial integrity.

## **Functions**

Functions in SQL Server are a powerful way to encapsulate reusable logic for calculations and data retrieval. The two types of functions that we will be covering in this course are:

- Scalar Functions- Returns a single value
- Table-Valued Functions- Returns a table

The prupose of functions are to simplify repetitive calculations so that your script is easier to understand. To start, we wil take a complex calculation from a previous assignment and turn it into a scalar function instead. Previously we had to calculate the subtotal of an order by doing (oi.item\_price - oi.discount\_amount) \* oi.quantity. We can delegate this operation to a function so that our SELECT statement looks a bit cleaner. To create a function we will need to learn a few new keywords.

- CREATE FUNCTION- Starts the script to create a function
- RETURNS- Describes what the function should return
- RETURN- The value that is being returned

In [ ]:
--Create scalar function to get subtotal
CREATE FUNCTION get_subtotal --Define and name the function
(
    @item_price DECIMAL(8,2), --List necessary parameters seperated by commas
    @discount_amount DECIMAL(8,2),
    @quantity INT
)
RETURNS DECIMAL(8,2) --Define the data type to return
AS
BEGIN
    RETURN (@item_price - @discount_amount) * @quantity; --Write the logic of the function
END;

The above function accepts three parameters, the item price, discount amount, and quantity since that is what it needs to know to calculate the subtotal. Now that the function has been defined, we can call it in a SELECT query just like we have other functions in the past, like GETDATE(). The only difference is that since this is not a built in function, we have to prefix it with dbo (database object).

In [ ]:
--Get order_item info with subtotal
SELECT *, dbo.get_subtotal(item_price, discount_amount, quantity) AS subtotal
FROM gs_order_items

We can see from the result in the above cell that we are calculating the subtotal as expected, and our SQL is much easier to read and understand. We call this process _abstraction._ Abstraction is the the process of taking a complex process and disguising it behind something simple. Up to this point we have had to be very verbose in our SQL scripts because we have not had the tools to abstract some of our more tedious processes, but now we can implement tools that make our scripts consice and secure.

  

Next we will work through putting together a Table-Valued Function, or TVF. In the spirit of abstration, we will continue to build onto the process that we began in the last set of queries. We will create a function that not only calculates the subtotal, but retrieves all of the data from the table when called. In addition, we will also allow a cuser to provide a value so that they can see all orders with a subtotal greater than the provided value. Note the differences in the below query to the scalar one we made earlier.

In [ ]:
--Get order items with subtotal above a certain amount
CREATE FUNCTION get_orders_over_subtotal --Define and name the function
(
    @subtotal_min DECIMAL(10,2) --Define required parameters
)
RETURNS TABLE --Specify the return type. For TVF it will always be TABLE
RETURN --Specify the return value
(
    SELECT *, dbo.get_subtotal(item_price, discount_amount, quantity) AS subtotal --Using the function/query we scripted before to get data
    FROM gs_order_items
    WHERE dbo.get_subtotal(item_price, discount_amount, quantity) > @subtotal_min --Using the function and comparing the result to parameter to filter data
)

Now we will call the function in a SELECT statement to see the result.

In [ ]:
--SELECT all records from the table returned by the function
--Orders with subtotal over 1000
SELECT *
FROM dbo.get_orders_over_subtotal(1000)

Custom functions that return tables can be extremely useful. If there is a subquery that you have to reference often, it may be better to build a function that secures that data and use that as a data source instead of a subquery. Both kinds of functions are highly custamizable because of the flexibility that parameters offer.

**Built in Functions**

To assist with your scripting, here are a list of built in functions and their definitions that may be useful moving forward:

GETDATE() - Returns the current date and time

DATEADD() - Adds a specified number of units to a date

DATEDIFF() - Returns the difference between two dates

DATEPART() - Returns a specific part of a date (e.g., year, month, day)

CAST() - Converts a value from one data type to another

CONVERT() - Converts a value from one data type to another with style formatting

ISNULL() - Replaces NULL with a specified replacement value

COALESCE() - Returns the first non-NULL value in a list of expressions

LEN() - Returns the number of characters in a string

SUBSTRING() - Extracts a substring from a string starting at a specified position

UPPER() - Converts a string to uppercase

LOWER() - Converts a string to lowercase

TRIM() - Removes leading and trailing spaces from a string

ROUND() - Rounds a numeric value to a specified number of decimal places

SUM() - Returns the sum of a set of values

AVG() - Returns the average of a set of values

MIN() - Returns the minimum value in a set

MAX() - Returns the maximum value in a set

COUNT() - Returns the number of items in a set

ROW\_NUMBER() - Returns the sequential number of a row within a result set

RANK() - Returns the rank of each row within the partition of a result set

LAG() - Provides access to a row at a specified physical offset before that row

LEAD() - Provides access to a row at a specified physical offset after that row

PATINDEX() - Returns the starting position of a pattern in a string

CHARINDEX() - Returns the position of a specified substring within a string

REPLACE() - Replaces all occurrences of a specified substring with another substring

## **Views**

A view in SQL Server is a virtual table based on the result set of an SQL query. It does not store data physically but provides a way to simplify complex queries and present a consistent, abstracted view of the underlying database schema. Views can be used to encapsulate logic and present data in a specific format without changing the underlying tables. Views have a handful of advantages:

1. Simplicity- Simplofy complex queries by encapsulating them in a view.
2. Security- Restrict access to specific data by granting permissions on views instead of the underlying tables.
3. Consistency- Provide a consistent interface to the data, even if the underlying schema changes
4. Reusability- Reuse complex queries across multiple applications or reports

We create views by using the CREATE VIEW keywords and then assigning a query to them using AS. In the below cell we will create a simple view that shows customers with the amount of orders they have placed. We will name the view gs\_vw\_customer\_info:

In [ ]:
--Create view that keeps track of basic customer info and number of orders
CREATE VIEW gs_vw_customer_info AS --define view and include vw in name to differentiate from tables
SELECT c.first_name, c.last_name, c.email_address,COUNT(o.order_id) AS total_orders
FROM gs_customers c
JOIN gs_orders o ON c.customer_id = o.customer_id
GROUP BY c.first_name, c.last_name, c.email_address

Now that we have created our view, we can access it's data like any other table:

In [ ]:
--Select all data from saved view
SELECT * FROM gs_vw_customer_info

Any SELECT query can be assigned to a view. Views are a great way to abstract complex calculations, subqueries, and window queries when needed. You can even use JOINs when referencing a view if needed. Views may sound similar to Table-Value Functions, and they are. Below are a few key differences:

- Parameters: TVFs can accept parameters; views cannot.
- Flexibility: TVFs are more flexible due to parameterization.
- Usage: Both can be used in SELECT statements, but TVFs can be used more dynamically.
- Optimization: TVFs can be optimized together with the calling query, while views provide a static result set.
- Definition: TVFs use a CREATE FUNCTION statement, whereas views use a CREATE VIEW statement.

## **Triggers**

A trigger in SQL Server is a special type of stored procedure that automatically executes or fires when certain events occur in the database. These events can be data manipulation operations such as INSERT, UPDATE, or DELETE on a table or view. Triggers are used to enforce business rules, maintain data integrity, and audit changes in the database. 

The most common use of triggers is for auditing purposes. We can create special tables to keep track of the changes in our production tables. We program triggers to 'fire' when a change to a production table is detected. We are going to create a simple trigger on the ingredient table that we made for the resteraunt system in lesson 9. Run the below cells to drop those tables if they exist, recreate them, and load them with data so that we are all working with the same info.

In [ ]:
--Drop resteraunt tables if they exist
DROP TABLE IF EXISTS re_recipe
DROP TABLE IF EXISTS re_ingredient
DROP TABLE IF EXISTS re_menu_item

In [ ]:
--Recreate tables
--Create ingregient table
CREATE TABLE re_ingredient (
    ingredient_id INT IDENTITY(1,1) PRIMARY KEY,
    ingredient_desc VARCHAR(50) NOT NULL,
    calories DECIMAL(6,2) DEFAULT 0
)

--Create menu table
CREATE TABLE re_menu_item (
    menu_item_id INT IDENTITY(1,1) PRIMARY KEY,
    menu_item_desc VARCHAR(60) NOT NULL,
    price DECIMAL(4,2) NOT NULL CHECK (price > 0)
)

--Create recipe table
CREATE TABLE re_recipe (
    recipe_id INT IDENTITY(1,1) PRIMARY KEY,
    ingredient_id INT NOT NULL,
    menu_item_id INT NOT NULL,
    quantity INT NOT NULL,
    CONSTRAINT fk_rec_ing_id_ref_ing_ing_id FOREIGN KEY (ingredient_id) REFERENCES re_ingredient(ingredient_id),
    CONSTRAINT fk_rec_men_id_ref_men_men_id FOREIGN KEY (menu_item_id) REFERENCES re_menu_item(menu_item_id)
)

--Assign index for menu item description
CREATE NONCLUSTERED INDEX IDX_menu_item_menu_item_desc ON re_menu_item(menu_item_desc)

In [ ]:
--Load tables with data
--Add data to ingredient
INSERT INTO re_ingredient VALUES
('Bun',140.00),
('Beef Patty', 200.00),
('Cheese', 50.00),
('Chicken Patty', 150.00),
('Buffalo Sauce', 20.00);

--Add data to menu
INSERT INTO re_menu_item VALUES
('Cheeseburger',2.99),
('Hamburger',1.75),
('Chicken Sandwich',2.99),
('Buffalo Chicken Sandwich', 3.19);

--Add data to recipe
INSERT INTO re_recipe VALUES
(1,1,1),
(2,1,1),
(3,1,1),
(1,2,1),
(2,2,1),
(1,3,1),
(4,3,1),
(1,4,1),
(4,4,1),
(5,4,1);

Now that we have some data to work with, we can begin the process of setting up our triggers by creating a table to store data about the changes. Review the below create statement and then run the cell.

In [ ]:
--Create table to log changes to re_menu_item
CREATE TABLE re_menu_item_changes (
    change_id INT IDENTITY(1,1) PRIMARY KEY,
    menu_item_id INT,
    change_type VARCHAR(50),
    change_date DATETIME DEFAULT GETDATE(), --Automatically get a timestamp of the change
    previous_data TEXT, --Store old data in JSON format
    new_data TEXT --Store new data in JSON format
);

Now that we have a place to store the information about the change, we can begin scripting the trigger that will capture that data. One of the functions of the trigger will be to consolidate the existing data and the updated data into a JSON format for easy storage. JSON is a text format that is good for storing a single row of data because the structure is similar to a standard object in programming. For more information about JSON check this link: [What is JSON?](https://www.w3schools.com/js/js_json_intro.asp)

In [ ]:
-- Create a DML trigger to log inserted records
CREATE TRIGGER trg_log_menu_item_changes_insert --Include insert in the name of the trigger since that is what makes it fire
ON re_menu_item --Which table does the trigger apply to
AFTER INSERT --Which operation should the trigger fire on
AS
BEGIN --Statement to run when trigger is fired
    INSERT INTO re_menu_item_changes VALUES
    (
        (
            SELECT i.menu_item_id --Get the menu_item_id that is being inserted using a subquery
            FROM inserted as i --inserted is a special temporary table that holds the values being inserted into the re_menu_item table
        ),
        'INSERT', --Specify that this is an insert operation
        GETDATE(), --Get the current date
        '', --There is no previous data since this is a fresh insertion, so this is blank
        (
            SELECT --This is a subquery that turns the data being inserted into a JSON format
            i.menu_item_id,
            i.menu_item_desc,
            i.price
            FROM inserted as i FOR JSON AUTO
        )
    );
END

Now that we have a trigger established, we can test it by inserting a new record into the re\_menu\_item table:

In [ ]:
--Add new record to re_menu_item
INSERT INTO re_menu_item VALUES
('French Fries',1.99);

The new data has been succesfully inserted. Note that there are two (1 row affected) outputs. One for the table we inserted to with our statement, and another for the one that our trigger added to. Let's check our log table to verify that everything worked correctly:

In [ ]:
SELECT * FROM re_menu_item_changes

It looks like everything went according to plan! We have:

-  a automatically generated change\_id
- the menu\_item\_id of the row that was inserted
- the type of change is listed as INSERT
- the change\_date shows the time of the insert
- there is nothing in previous data since there was no existing data for menu\_item\_id 5 before this
- most interstingly, we have a JSON preservation of the data that was added to the table. If something ever went wrong we could use this JSON to restore the data.

Now that we have a feel for how triggers work, let's make a delete trigger

In [ ]:
-- Create a DML trigger to log inserted deleted
CREATE TRIGGER trg_log_menu_item_changes_delete --include delete in the trigger name
ON re_menu_item
AFTER DELETE --specify that it should fire when a deletion occurs in the above table
AS
BEGIN --Statement to run on fire
    INSERT INTO re_menu_item_changes VALUES
    (
        (
            SELECT d.menu_item_id
            FROM deleted as d --Just like inserted, deleted is a temporary table that holds the data that is about to be removed
        ),
        'DELETE',
        GETDATE(),
        (
            SELECT 
            d.menu_item_id,
            d.menu_item_desc,
            d.price
            FROM deleted as d FOR JSON AUTO
        ),
        '' --We don't have any new data since this is a deletion, so this is blank
    )
END

Let's go ahead and test our trigger by deleting the data that we just inserted:

In [ ]:
--Delete data from re_menu_item
DELETE FROM re_menu_item
WHERE menu_item_id = 5

The deletion was succesful, let's check the log table to verify that it captured our changes:

In [ ]:
--Check log
SELECT * FROM re_menu_item_changes

We can see that the deletion operation wrote to the log table as expected. In a later unit I will show you how to restore delete data as needed. Next we are going to write a trigger to log updates to the menu item table. In the instance of an update trigger being fired the old data is stored in the temporary table _deleted_ and the new data is stored in the temporary table _inserted._

In [ ]:
-- Create a DML trigger to log updated records
CREATE TRIGGER trg_log_menu_item_changes_update --Include update in the name
ON re_menu_item
AFTER UPDATE --Specify that trigger should fire when update is called on the above table
AS
BEGIN --Script to run when fired
    INSERT INTO re_menu_item_changes VALUES
    (
        (
            SELECT d.menu_item_id
            FROM deleted as d
        ),
        'UPDATE',
        GETDATE(),
        ( --Select old data by referencing deleted
        SELECT 
            d.menu_item_id,
            d.menu_item_desc,
            d.price
            FROM deleted as d FOR JSON AUTO
        ),
        ( --Select new data by referencing inserted
            SELECT 
            i.menu_item_id,
            i.menu_item_desc,
            i.price
            FROM inserted as i FOR JSON AUTO
        )
    )
END

In [ ]:
--Update a record to test the trigger
UPDATE re_menu_item SET price = 3.14
WHERE menu_item_id = 1

In [ ]:
--Check the change log to verify the change was recorded
SELECT * FROM re_menu_item_changes

## **Exercises/Scenario**

For this scenario set you will be creating stored procedures, functions, views, and triggers to improve the efficency of the Airport database. The ERD has been included in github for your convenience. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Remember that none of the airport tables have IDENTITY insert for primary keys.</span>

1\. Your first task is to create a log table for the boarding pass table. It should be similar in structure and naming convention to the log table that we made earlier in the notebook.

In [2]:
--Your code here
CREATE TABLE ap_boarding_pass_log (
    log_id INT PRIMARY KEY,
    boarding_pass_id INT,
    customer_id INT,
    flight_id INT,
    boarding_pass_number VARCHAR(19),
    operation_type VARCHAR(10),
    operation_time DATETIME
);



Commands completed successfully.

Total execution time: 00:00:00.086

2\. In the below cell, construct a trigger that is fired upon an update to the boarding pass table. This trigger should capture both the data before and after the update in a JSON format.

In [3]:
--Your code here
CREATE TRIGGER trg_ap_boarding_pass_update
ON ap_boarding_pass
AFTER UPDATE
AS
BEGIN
    DECLARE @old_data NVARCHAR(MAX), @new_data NVARCHAR(MAX);

    SELECT @old_data = (
        SELECT * FROM deleted FOR JSON AUTO
    );

    SELECT @new_data = (
        SELECT * FROM inserted FOR JSON AUTO
    );

    INSERT INTO ap_change_log (entity, old_data, new_data, change_time)
    VALUES ('ap_boarding_pass', @old_data, @new_data, GETDATE());
END;



Commands completed successfully.

Total execution time: 00:00:00.120

3\. In the below cell, create a stored procedure (get\_flight\_data) that returns all of the data in the flight table where destination matches the parameter @input\_destination.

In [4]:
--Your code here
CREATE PROCEDURE ap_get_flight_data
    @input_destination VARCHAR(20)
AS
BEGIN
    SELECT *
    FROM ap_flight
    WHERE destination = @input_destination;
END;



Commands completed successfully.

Total execution time: 00:00:00.078

4\. In the below cell, execute the stored procedure you created in the previous cell to return information for flights going to Tokyo.

In [5]:
--Your code here
EXEC ap_get_flight_data @input_destination = 'Tokyo';



(0 rows affected)

Total execution time: 00:00:00.122

flight_id,airline_id,flight_number,destination


5\. In the below cell, create a view (ap\_vw\_airline\_flights) that shows the airline\_id, airline\_name, and total number of flights for each airline.

In [6]:
--Your code here
CREATE VIEW ap_vw_airline_flights AS
SELECT 
    a.airline_id, 
    a.airline_name, 
    COUNT(f.flight_id) AS total_flights
FROM 
    ap_airline a
JOIN 
    ap_flight f ON a.airline_id = f.airline_id
GROUP BY 
    a.airline_id, a.airline_name;



Commands completed successfully.

Total execution time: 00:00:00.083

6\. In the below cell, run a SELECT query to show all data from the view in the previous cell.

In [7]:
--Your code here
SELECT * 
FROM ap_vw_airline_flights;


(3 rows affected)

Total execution time: 00:00:00.228

airline_id,airline_name,total_flights
1,FlyHigh Airlines,2
2,SkyReach Airways,2
3,Global Wings,1


7\. In the below cell, create a function (get\_age) that returns the age of a customer when provided the date\_of\_birth.

In [8]:
--Your code here
CREATE FUNCTION ap_get_age (@dob DATE)
RETURNS INT
AS
BEGIN
    RETURN DATEDIFF(YEAR, @dob, GETDATE());
END;




Commands completed successfully.

Total execution time: 00:00:00.107

8\. In the below cell, write a select statement to query all information for every customer. Incorporate the function from the above cell to also include their age.

In [1]:
--Your code here
SELECT 
    c.*, 
    dbo.ap_get_age(c.date_of_birth) AS age
FROM 
    ap_customer c;


: Msg 208, Level 16, State 1, Line 2
Invalid object name 'customer'.

Total execution time: 00:00:00.079